In [3]:
preparser(False)

In [12]:
R = PolynomialRing(QQ, ['x1', 'x2'])
x1, x2 = R.gens()

A = matrix(QQ, [[0, -1], [1, 0]])  # 90° rotation matrix
t = vector(QQ, [1, 1])            # translation vector

def S2(v): return A * v
def w2(v): return - v + t

v = vector([x1, x2])



def S2w2(v): return S2(w2(v))

S2w2_2 = S2w2(S2w2(v))

print("S2^4(v) =", S2(S2(S2(S2(v)))))
print("w2^2(v) =", w2(w2(v)))
print("(w2*S2)^4(v) =", w2(S2(w2(S2(w2(S2(w2(S2(v)))))))))
print("w2*S2^2", w2(S2(S2(v))))

S2^4(v) = (x1, x2)
w2^2(v) = (x1, x2)
(w2*S2)^4(v) = (x1, x2)
w2*S2^2 (x1 + 1, x2 + 1)


In [13]:
def B0_as_finitely_presented_group(N):
    """
    Construct the subgroup B_0 of the automorphism group of X_0(N) as finitely presented group. 
    Returns (B_0, invariants) and invariants is a dictionary containing v2, v3, mu, omega, v.
    
    Warning!!! At the moment this funtion returns some relations that are not proven to hold if v2>=5.
    Verify that the returned relations are satisfied before using this function in a proof.
    """
    N = Integer(N)
    if N <= 0:
        raise ValueError(f"N={N} must be a positive integer.")

    fac = N.factor()   # list of (prime, exponent)
    primes = N.prime_divisors()

    v2 = Integer(N).valuation(2)
    v3 = Integer(N).valuation(3)

    mu = min(3, v2 // 2)       
    omega = min(1, v3 // 2)
    v_val = 2**mu * 3**omega

    names = []
    # Build generator list: S2, S3, and w_p for each prime dividing N
    if 2 in primes:
        names = ["w2", "S2"]
    if 3 in primes:
        names += ["w3", "S3"]
    names += ["w{}".format(p) for p in primes if p > 3]
    F = FreeGroup(len(names),names)
    gens = list(F.gens())
    gens_map = F.gens_dict()

    if 2 in primes:
        S2 = gens_map["S2"]
    if 3 in primes:
        S3 = gens_map["S3"]
    

    rels = []

    # --- all w_p of order 2 and commute pairwise ---
    for p in primes:
        wp = gens_map[f"w{p}"]
        rels.append(wp**2)
    for i in range(len(primes)):
        for j in range(i+1, len(primes)):
            p, q = primes[i], primes[j]
            wp, wq = gens_map[f"w{p}"], gens_map[f"w{q}"]
            rels.append(wp * wq * (wq * wp)**(-1))

    # (S2)^{2^mu} = 1, (S3)^{3^omega} = 1
    if 2 in primes:
        rels.append(S2 ** (2**mu))
    if 3 in primes:
        rels.append(S3 ** (3**omega))

    # --- 3-part relations ---
    if v3 == 2:
        rels.append((gens_map["w3"] * S3) ** 3)
    if v3 >= 3:
        a = gens_map["w3"] * S3 * gens_map["w3"]
        rels.append(a * S3 * (S3 * a)**(-1))

    # --- 2-part relations ---
    if "w2" in gens_map:
        w2 = gens_map["w2"]
        if v2 in (2, 4, 6):
            # second relation is unproven at the moment if v2=6
            rels += [(w2 * S2)**3, w2 * S2**(-3) * w2 * S2**(-2) * w2 * S2 * w2 * S2**(-2)]
        elif v2 in (3, 5, 7):
            # second relation is unproven at the moment if v2=5,7
            # it can be replaced with the aslo unproven #(w2 * S2**(-1) * w2 * S2)**2 if v2=5
            rels += [(w2 * S2)**4, w2 * S2**3 * w2 * S2**2 * w2 * S2 * w2 * S2**(-2)]
        elif v2 == 8:
            rels.append((w2 * S2)**2 * (w2 * (S2**3))**2)
        elif v2 >= 9:
            b = w2 * S2 * w2
            rels.append(b * S2 * (S2 * b)**(-1))

    # S2 and S3 commute
    if 2 in primes and 3 in primes:
        rels.append(S2 * S3 * (S3 * S2)**(-1))

    # --- generic relations for w_p with S2, S3 ---
    for p in primes:
        wp = gens_map[f"w{p}"]
        pe = p**(N.valuation(p))
        if p != 2 and 2 in primes:
            rels.append(wp * S2 * wp * S2 ** (-pe%(2**mu)))
        if p != 3 and 3 in primes:
            rels.append(wp * S3 * wp * S3 ** (-pe%(3**omega)))

    # quotient
    B0 = F / rels

    invariants = {"v2": int(v2), "v3": int(v3), "mu": int(mu),
                  "omega": int(omega), "v": int(v_val), "primes": primes}

    return B0, invariants
    
    
def apply(m,g,ambient=False):
    g = list(g)
    M = m.parent()
    if ambient:
        M = M.ambient()
    gm = 0
    for a,c in m.modular_symbol_rep():
        gm += a*c.apply(g)
    return M(gm)

def atkin_lehner_divisors(N):
    al_divisors = []
    for d in divisors(N):
        if gcd(d, N // d) == 1:
            al_divisors.append(d)
    return al_divisors

def Sv_operator(S, v):
    assert 24%v == 0
    assert S.level()%(v**2)==0
    g = [v,1,0,v]
    rows = [S.coordinate_vector(apply(m,g)) for m in S.basis()]
    return Matrix(rows)

def cusp_permutation(G, m):
    C = G.cusps()
    if not isinstance(m,list):
        m = m.list()
    perm = [C.index(G.reduce_cusp(c.apply(m)))+1 for c in C]
    return Permutation(perm)

def group_relations(G, names):
    """Return the relations between the generators of the permuation group G
    the list given in names will be used as names for the variables in these relations.
    
    Warning!!!  If you did not create the permutation group G with canonicalize=False then
    the generators of G might not be what you expect them to be.
    """
    assert G.ngens() == len(names)
    F = FreeGroup(names)
    Gfp = G.as_finitely_presented_group()
    assert all([Gfp.gen(i)==Gfp(G.gen(i)) for i in range(G.ngens())])
    generator_mapping = {str(Gfp.gen(i)):F.gen(i) for i in range(G.ngens())}
    return [rel.subs(**generator_mapping) for rel in Gfp.relations()]
    
def E0_as_permutation_group(N, full_level=False):
    """Return the image of E_0 as acting on the cusps.
    A priori this could be a quotient of E_0. However in practice this 
    permutation representation is often faithfull.
    """
    N = ZZ(N)
    v2 = Integer(N).valuation(2)
    v3 = Integer(N).valuation(3)

    mu = min(3, v2 // 2)       
    omega = min(1, v3 // 2)
    
    if v2==0 and v3 == 0:
        return PermutationGroup([]),[]
    if full_level:
        G = Gamma0(N)
    else:
        G = Gamma0(2**v2*3**v3)

    gens = []
    names = []
    if v2 > 0: 
        w2 = cusp_permutation(G,G.atkin_lehner_matrix(2))
        S2 = cusp_permutation(G,[2**mu,1,0,2**mu])
        gens = [w2, S2]
        names = ["w2", "S2"]
    if v3 > 0:
        w3 = cusp_permutation(G,G.atkin_lehner_matrix(3))
        S3 = cusp_permutation(G,[3**omega,1,0,3**omega])
        gens += [w3, S3]
        names += ["w3", "S3"]
    return PermutationGroup(gens, canonicalize=False), names
    
def B0_as_permutation_group(N):
    N = ZZ(N)
    primes = [p for p in N.prime_divisors() if p > 3]
    AL_operators = tuple(cusp_permutation(G, G.atkin_lehner_matrix(p)) for p in primes)
    E0, names = E0_as_permutation_group(N, full_level=True)
    names += [f"w{p}" for p in primes]
    return PermutationGroup(E0.gens()+AL_operators, canonicalize=False), names
   

def E0_relations_on_cusps(N, full_level=False):
    E0, names = E0_as_permutation_group(N, full_level=full_level)
    return group_relations(E0, names)
    

def E0_as_matrix_group(N, weight=2, cuspidal=True):
    G = Gamma0(N)
    M = ModularSymbols(G, weight=weight)
    if cuspidal:
        S = M.cuspidal_submodule()
    else:
        S = M
        
    v2 = Integer(N).valuation(2)
    v3 = Integer(N).valuation(3)

    mu = min(3, v2 // 2)       
    omega = min(1, v3 // 2)
    gens = []
    names = []
    
    if v2 > 0: 
        w2 = S.atkin_lehner_operator(2).matrix()
        S2 = Sv_operator(S, 2**mu)
        gens = [w2, S2]
        names = ["w2", "S2"]
    if v3 > 0:
        w3 = S.atkin_lehner_operator(3).matrix()
        S3 = Sv_operator(S, 3**omega)
        gens += [w3, S3]
        names += ["w3", "S3"]
    return MatrixGroup(gens), names
    
def B0_as_matrix_group(N, weight=2, cuspidal=True):
    N = ZZ(N)
    G = Gamma0(N)
    M = ModularSymbols(G, weight=weight)
    
    if cuspidal:
        S = M.cuspidal_submodule()
    else:
        S = M
        

    primes = [p for p in N.prime_divisors() if p > 3]
    AL_operators = tuple(S.atkin_lehner_operator(p).matrix() for p in primes)
    E0, names = E0_as_matrix_group(N, weight=weight, cuspidal=cuspidal)
    names += [f"w{p}" for p in primes]
    return MatrixGroup(E0.gens()+AL_operators), names
   

In [14]:
E0, names = E0_as_permutation_group(2**8)
w2, S2 = E0.gens()
w2*S2**2*w2*S2==S2**2*w2*S2*w2

False

In [15]:
todo = [140, 180, 220, 252, 280, 288, 360, 440, 720 ]
for N in todo:
    B0, names =  B0_as_matrix_group(N)
    ZB0 = B0.center()
    print(N, B0.cardinality(), ZB0.cardinality(),ZB0.exponent())
    genera = [(z.matrix()-1).kernel().dimension()/2 for z in ZB0 if z.matrix()!=1]
    print(f"genera: {genera}")

140 24 4 2
genera: [10, 10, 7]


180 144 1 1
genera: []


220 24 4 2
genera: [16, 13, 10]


252 144 2 2
genera: [19]


280 32 8 2


...

288 384 2 2
genera: [17]


360 192 2 2
genera: [29]


440 32 8 2


genera: [33, 33, 25, 33, 33, 33, 33]


720 576 1 1
genera: []


In [16]:
todo =  [140, 180, 220, 252, 280, 288, 360, 440, 720 ]
for N in todo:
    B0, names =  B0_as_matrix_group(N)
    #ZB0 = B0.center()
    print(f"level: {N}, #B0({N})={B0.cardinality()}")
    genera = [(z.matrix()-1).kernel().dimension()/2 for z in B0 if z!=B0(1) and z**2==B0(1)]
    print(f"minimum  {min(genera)}, all genera: {genera}")

level: 140, #B0(140)=24


minimum  7, all genera: [10, 10, 7, 9, 8, 10, 7, 9, 8, 10, 7, 9, 8, 10, 7]


level: 180, #B0(180)=144


minimum  11, all genera: [11, 11, 11, 13, 11, 13, 11, 13, 11, 11, 11, 11, 11, 11, 11, 13, 11, 11, 11, 13, 11, 11, 11, 13, 11, 11, 11, 13, 11, 11, 11, 13, 11, 11, 11, 13, 11, 11, 11]


level: 220, #B0(220)=24


...

minimum  25, all genera: [25, 29, 29, 29, 29, 29, 29, 25, 29, 29, 25, 29, 29, 29, 29, 25, 29, 29, 29, 29, 25, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 25, 25, 25, 29, 25, 29, 25, 29, 29, 29, 25, 29, 29, 25, 29, 25, 29, 25, 29, 25, 25, 25, 29]


level: 440, #B0(440)=32


minimum  25, all genera: [33, 33, 25, 31, 33, 27, 29, 33, 31, 33, 27, 33, 31, 33, 27, 31, 33, 27, 29, 33, 33, 33, 33]


level: 720, #B0(720)=576


minimum  57, all genera: [61, 61, 57, 61, 57, 57, 61, 61, 61, 57, 57, 61, 57, 61, 57, 61, 57, 61, 57, 61, 57, 61, 57, 61, 57, 61, 57, 61, 57, 61, 57, 61, 57, 61, 57, 61, 57, 61, 57, 61, 57, 57, 57, 57, 61, 61, 57, 61, 61, 57, 61, 61, 61, 57, 57, 61, 57, 61, 57, 57, 57, 57, 57, 61, 57, 57, 61, 57, 57, 61, 57, 57, 61, 57, 57, 61, 57, 57, 61, 57, 57, 61, 57, 57, 61, 57, 57, 61, 61, 57, 57, 57, 57, 61, 61, 57, 57, 57, 57]
